In [3]:
import time
import pickle
import pandas as pd
import numpy as np
import pandas as pd
import math
import copy

from sklearn.mixture import GaussianMixture as GM
from scipy.optimize import minimize
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from sklearn.metrics import davies_bouldin_score as DBS


from matplotlib import style

import seaborn as sns
###################
from sklearn.cluster import AgglomerativeClustering as AggClust
#from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn_extra.cluster import KMedoids

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [13]:
#test previous code:

#obj_daily, storage_profit_daily, wind_curtailment_daily, wind_notcurtailed_daily, storage_op_dict, dict_ess_profile, dict_details, obj_cost_breakdown = storage_included_uc(skip_LMP=True, ess_flag=False)

In [5]:
import copy

def reduce_df_to_one_row(my_df, row_idx):
    return my_df.iloc[[row_idx]]

def storage_included_uc_24bus(skip_LMP=True,ess_flag=False, folder_main="data_original\\", 
                              day_idx=None, curt_cost=500,
                             add_noise=False):
    ###
    
    """
    Unit commitment for 365 days for the 3-bus test system
    """
    
    Demand = 2650 
    parent_path = "E:\\second_paper\\Code_and_Data\\paper2_data_input\\" 
    non_system_data_path = parent_path + folder_main
    
    ## IMPORT DATA ##
    df_load_profile = pd.read_csv(non_system_data_path + "Load.csv") 
    
    if add_noise:
        seed = int(np.sqrt(abs(hash('Load'))))
        np.random.seed(seed)
        noise = (10 * np.random.rand(*df_load_profile.shape) - 5)/50
        df_load_profile = df_load_profile + noise
        df_load_profile = np.clip(df_load_profile, a_min=0, a_max=1)

    
    if day_idx is not None:
        df_load_profile = reduce_df_to_one_row(df_load_profile, day_idx)
    

    #should be removed!
    #df_wind_profile = pd.read_csv("InputData_Wind.csv")
    #df_wind_profile_extra = pd.read_csv("InputData_Wind_extra.csv")
    #df_wind_profile_extra_bus1 = pd.read_csv("InputData_Wind_extra_bus1.csv")
    
    #df_wind_profile_dict={}
    #df_wind_profile_dict[1] = df_wind_profile_extra_bus1
    #df_wind_profile_dict[2] = df_wind_profile
    #df_wind_profile_dict[3] = df_wind_profile
    #df_wind_profile_dict[4] = df_wind_profile_extra
    
    

    df = pd.read_excel(parent_path + 'System_Data_24_bus.xlsx', sheet_name=None, engine='openpyxl')

    
    ## SETTINGS ##
    UF=0;
    DF=2;
    VOLL=1e6;
    Curt= curt_cost;  #50 was good for k=10, (also ESS P30E150), also k=2
    
    #print("df['GDATA']['genNo.']: \n", df['GDATA']['genNo.']);
    
    gen_set=list(set(df['GDATA']['genNo.']));
    #print('gen_set: ', gen_set);
    
    bus_set = list(set(df['CapD']['bus']));
    #print('bus_set: ', bus_set)
    
    lin_set = list(set(df['LDATA']['lineNo.']));
    #print('lin_set: ', lin_set)
    
    t_set = list(range(1,df_load_profile.shape[1]));  #why not .shape[]+1
    d_set = list(range(1,df_load_profile.shape[0]+1));
    stor_set = list(set(df['SDATA']['storNo.']))
    
    #print('>>>')
    #print("df['MapG'].to_numpy() \n", df['MapG'].to_numpy())
    #print('<<<')
    
    #print('\n t_set: ', t_set)

    GDATA = df['GDATA'].set_index(['genNo.']).to_dict()
    LDATA = df['LDATA'].set_index(['lineNo.']).to_dict()
    SDATA = df['SDATA'].set_index(['storNo.']).to_dict()
    


    MapG={}
    MapL={}
    MapS={}


    for b in range(len(bus_set)):
        for g in range(len(gen_set)):
            MapG[bus_set[b],gen_set[g]] = df['MapG'].to_numpy()[g][b]
        for l in range(len(lin_set)):
            MapL[bus_set[b],lin_set[l]] = df['MapL'].to_numpy()[l][b]

        for s in range(len(stor_set)):
            MapS[bus_set[b],stor_set[s]] = df['MapS'].to_numpy()[s][b]


    #print('\n MapG: \n', MapG)
    #print('-------------')
    #print('\n MapL: \n', MapL)


    P_wind={}
    P_load={}
    for b in range(len(bus_set)):
        #print('bus (as index) is: ', b)
        
        ###if df['CapW']['Wind'].to_numpy()[b]>0:
        ###   name_of_RE = df['CapW'].loc[b,'Profile']
        ######print('name_of_RE: ', name_of_RE)
        
        if df['CapW']['Wind'].to_numpy()[b]>0: #else: means there is no wf.. and so...no name_of_RE
            name_of_RE = df['CapW'].loc[b,'Profile']
            df_wind_profile = pd.read_csv(non_system_data_path + (name_of_RE + ".csv"))
        
            if add_noise:
                seed = int(np.sqrt(abs(hash(name_of_RE))))
                np.random.seed(seed)
                noise = (10 * np.random.rand(*df_wind_profile.shape) - 5)/50
                df_wind_profile = df_wind_profile + noise
                df_wind_profile = np.clip(df_wind_profile, a_min=0, a_max=1)

        
        for t in range(len(t_set)):
            for d in range(len(d_set)):
                
                #P_wind[bus_set[b],t_set[t],d_set[d]]=df['CapW']['Wind'].to_numpy()[b]*df_wind_profile.to_numpy()[d][t+1]
                
                if df['CapW']['Wind'].to_numpy()[b]==0:
                    P_wind[bus_set[b],t_set[t],d_set[d]] = 0
                else:
                    df_wind_profile_to_use = df_wind_profile.copy()
                    if day_idx is not None:
                        df_wind_profile_to_use = reduce_df_to_one_row(df_wind_profile_to_use, day_idx)
                    
                
                    P_wind[bus_set[b],t_set[t],d_set[d]]=df['CapW']['Wind'].to_numpy()[b] * df_wind_profile_to_use.to_numpy()[d][t+1]
                
                P_load[bus_set[b],t_set[t],d_set[d]]=((0.01 * df['CapD']['Load_Percentage'].to_numpy()[b]) * Demand) * df_load_profile.to_numpy()[d][t+1]



                
                
                
    #print('-----')
    #print('>>> P_wind: ', P_wind)
    #print('-----')



    model=pe.ConcreteModel()

    #Variables

    # >>> p line, voltage angles of buses, and p gen
    model.PL = pe.Var(lin_set,t_set,d_set,domain=pe.Reals)
    model.theta = pe.Var(bus_set,t_set,d_set,domain=pe.Reals)
    model.PG = pe.Var(gen_set,t_set,d_set,domain=pe.NonNegativeReals)

    # >>> Loadloss and Curtailment variables
    model.PLS = pe.Var(bus_set,t_set,d_set,domain=pe.NonNegativeReals)
    model.Pcurt = pe.Var(bus_set,t_set,d_set,domain=pe.NonNegativeReals)

    # >>>  on/off, start, shutdown variables of gen
    model.w = pe.Var(gen_set,t_set,d_set,domain=pe.Binary)
    model.u = pe.Var(gen_set,t_set,d_set,domain=pe.Binary)
    model.f = pe.Var(gen_set,t_set,d_set,domain=pe.Binary)

    
    if ess_flag:
        # >>> storage variables
        model.pch = pe.Var(stor_set,t_set,d_set,domain=pe.NonNegativeReals)
        model.ifch = pe.Var(stor_set,t_set,d_set,domain=pe.Binary)
        model.pdch = pe.Var(stor_set,t_set,d_set,domain=pe.NonNegativeReals)
        model.ifdch = pe.Var(stor_set,t_set,d_set,domain=pe.Binary)
        model.soc = pe.Var(stor_set,t_set,d_set,domain=pe.NonNegativeReals)







    ## objective function: minimize operation cost
    
    
    
    
    def obj_expression(model):
        return sum(
              sum(
                sum((model.PG[g,t,e]*GDATA['Cg'][g]+model.w[g,t,e]*GDATA['Noload'][g]+
                     model.u[g,t,e]*GDATA['StartUp'][g]+model.f[g,t,e]*GDATA['ShutDn'][g]) for g in gen_set)+
                sum(model.PLS[b,t,e]*VOLL for b in bus_set)+
                sum(Curt*model.Pcurt[b,t,e] for b in bus_set)
            for t in t_set)
                for e in d_set)
    
    
    model.OBJ = pe.Objective(rule=obj_expression, sense=pe.minimize)

    #  constraint: Power balance constraint
    
    if ess_flag:
        def Lambda_rule(model,b,t,e):
            return (sum(model.PG[g,t,e]*MapG[b,g]
                       for g in gen_set)-sum(model.PL[l,t,e]*MapL[b,l]
                                             for l in lin_set) -  
                                            sum(model.pch[s,t,e]*MapS[b,s]
                                             for s in stor_set) +  
                                            sum(model.pdch[s,t,e]*MapS[b,s]
                                             for s in stor_set) + model.PLS[b,t,e]+(P_wind[b,t,e]-model.Pcurt[b,t,e])==P_load[b,t,e])
    
    else:
        
        def Lambda_rule(model,b,t,e):
            return sum(model.PG[g,t,e]*MapG[b,g]
                       for g in gen_set)-sum(model.PL[l,t,e]*MapL[b,l]
                                             for l in lin_set) + model.PLS[b,t,e]+(P_wind[b,t,e]-model.Pcurt[b,t,e])==P_load[b,t,e]
        
    
    model.Lambda = pe.Constraint(bus_set, t_set, d_set, rule=Lambda_rule)


    # constraint: Line Flow constraints
    def Delta_rule(model,l,t,e):          
        return model.PL[l,t,e]==100*LDATA['B'][l]*sum(MapL[b,l]*model.theta[b,t,e] for b in bus_set)

    def Gamma_Plus_rule(model,l,t,e):          
        return model.PL[l,t,e]<=LDATA['Flmax'][l]

    def Gamma_Minus_rule(model,l,t,e):          
        return model.PL[l,t,e]>=-LDATA['Flmax'][l]

    model.Delta = pe.Constraint(lin_set, t_set, d_set, rule=Delta_rule)
    model.Gamma_Plus = pe.Constraint(lin_set, t_set, d_set, rule=Gamma_Plus_rule)
    model.Gamma_Minus = pe.Constraint(lin_set, t_set, d_set, rule=Gamma_Minus_rule)


    if ess_flag:
        # constraint: Storage constraints
        def Charging_rule(model,s,t,e):
            return model.pch[s,t,e] <= model.ifch[s,t,e]*SDATA['Ps'][s]


        def Discharging_rule(model,s,t,e):
            return model.pdch[s,t,e] <=  model.ifdch[s,t,e]*SDATA['Ps'][s]

        def vote_rule(model,s,t,e):
            return (model.ifch[s,t,e] + model.ifdch[s,t,e] <= 1)


        def SOClimitmin_rule(model,s,t,e):
            if t==24:
                return (model.soc[s,t,e] ==  SDATA['SOCfin'][s]*SDATA['Es'][s])

            else:
                return (model.soc[s,t,e]>=SDATA['SOCmin'][s]*SDATA['Es'][s])


        def SOClimitmax_rule(model,s,t,e):
            if t==24:
                return (model.soc[s,t,e] ==  SDATA['SOCfin'][s]*SDATA['Es'][s])

            else:
                return (model.soc[s,t,e] <= SDATA['SOCmax'][s]*SDATA['Es'][s])



        def SOC_rule(model,s,t,e):
            if t==1:
                return model.soc[s,t,e] == SDATA['SOCinit'][s]*SDATA['Es'][s] + (SDATA['eff'][s]*model.pch[s,t,e] - (1/SDATA['eff'][s])*model.pdch[s,t,e])

            else:
                return model.soc[s,t,e] == model.soc[s,t-1,e] + (SDATA['eff'][s]*model.pch[s,t,e] - (1/SDATA['eff'][s])*model.pdch[s,t,e])

        
        model.Charging = pe.Constraint(stor_set, t_set, d_set, rule=Charging_rule)
        model.Discharging = pe.Constraint(stor_set, t_set, d_set, rule=Discharging_rule)
        model.vote = pe.Constraint(stor_set, t_set, d_set, rule=vote_rule)
        model.SOClimitmin = pe.Constraint(stor_set, t_set, d_set, rule=SOClimitmin_rule)
        model.SOClimitmax = pe.Constraint(stor_set, t_set, d_set, rule=SOClimitmax_rule)
        model.SOC = pe.Constraint(stor_set, t_set, d_set, rule=SOC_rule)


    # constraint: Genrators constraints
    def Mu_Plus_rule(model,g,t,e):          
        return model.PG[g,t,e]<=model.w[g,t,e]*GDATA['Pgmax'][g]

    def Mu_Minus_rule(model,g,t,e):          
        return model.PG[g,t,e]>=model.w[g,t,e]*GDATA['Pgmin'][g]


    ###############
    ###############


    def Ramp_Up_rule(model,g,t,e):
        if t<=23:
            return model.PG[g,t+1,e]-model.PG[g,t,e]<=GDATA['Ramp'][g]
        else:
            return -model.PG[g,t,e]<=GDATA['Ramp'][g]

    def Ramp_Dn_rule(model,g,t,e):          
        if t<=23:
            return model.PG[g,t,e]-model.PG[g,t+1,e]<=GDATA['Ramp'][g]
        else:
            return -model.PG[g,t,e]<=GDATA['Ramp'][g] ######missing - sign? (should be true always)

    #############
    #############

    def Min_Up_rule(model,g,t,e):
        if (t>=UF+1):
            if t-GDATA['MinUp'][g]+1>=1:
                return sum(model.u[g,k,e] for k in range(t-GDATA['MinUp'][g]+1, t)) <=model.w[g,t,e]
            else:
                return model.w[g,t,e]>=0;
        else:
            return model.w[g,t,e]>=0;

    def Min_Dn_rule(model,g,t,e):
        if (t>=DF+1):
            if t-GDATA['MinDn'][g]+1>=1:
                return sum(model.f[g,k,e] for k in range(t-GDATA['MinDn'][g]+1, t)) <=1-model.w[g,t,e]
            else:
                return model.w[g,t,e]>=0;
        else:
            return model.w[g,t,e]>=0;


    #############    
    #############

    def Coordinate_rule(model,g,t,e):
        if t>=2:
            return model.w[g,t-1,e]-model.w[g,t,e]+model.u[g,t,e]-model.f[g,t,e]==0
        else:
            return -model.w[g,t,e]+model.u[g,t,e]-model.f[g,t,e]==0


    def Start_Shut_rule(model,g,t,e):
        return model.u[g,t,e]+model.f[g,t,e]<=1


    model.Mu_Plus = pe.Constraint(gen_set, t_set, d_set, rule=Mu_Plus_rule)
    model.Mu_Minus = pe.Constraint(gen_set, t_set, d_set, rule=Mu_Minus_rule)
    model.Ramp_Up = pe.Constraint(gen_set, t_set, d_set, rule=Ramp_Up_rule)
    model.Ramp_Dn = pe.Constraint(gen_set, t_set, d_set, rule=Ramp_Dn_rule)
    model.Min_Up = pe.Constraint(gen_set,t_set, d_set, rule=Min_Up_rule)
    model.Min_Dn = pe.Constraint(gen_set, t_set, d_set, rule=Min_Dn_rule)
    model.Coordinate = pe.Constraint(gen_set, t_set, d_set,rule=Coordinate_rule)
    model.Start_Shut = pe.Constraint(gen_set, t_set, d_set,rule=Start_Shut_rule)




    






     # constraint: Load Shedding constraint
    def Rho_Plus_rule(model,b,t,e):          
        return model.PLS[b,t,e]<=P_load[b,t,e]

    model.Rho_Plus = pe.Constraint(bus_set, t_set, d_set, rule=Rho_Plus_rule)

     # constraint: Angle constraints
    def Eta_Ref_rule(model,b,t,e):          
        if b==1:
            return model.theta[b,t,e]==0
        else:
            return model.theta[b,t,e]<=3.14

    model.Eta_Ref = pe.Constraint(bus_set, t_set, d_set, rule=Eta_Ref_rule)


     # constraint: Wind curtailment constraints
    def WindCurlim_rule(model,b,t,e):
        return model.Pcurt[b,t,e]<=P_wind[b,t,e]                    

    model.WindCurlim = pe.Constraint(bus_set, t_set, d_set, rule=WindCurlim_rule)

    # SOLVE ##

    #model.dual = pe.Suffix(direction=pe.Suffix.IMPORT)



    opt = pe.SolverFactory('cplex')

    results = opt.solve(model, tee=False) #tee=True (to show the script as being run)

    
    


    #############print(results)
    #############print("\n >>> (first round) Objective = ", model.OBJ())


    #print("\nValue of Total Wind Curtailment (MWh) = ", sum(sum(sum(model.Pcurt[b,t,e].value for b in bus_set) for t in t_set) for e in d_set))

    #print("\nValue of Total Load shedding (MWh) = ", sum(sum(sum(model.PLS[b,t,e].value for b in bus_set) for t in t_set) for e in d_set))

    
    
    
    #############################################
    #############################################
    
    w_fixed = {}
    u_fixed = {}
    f_fixed = {}

    for g in range(len(gen_set)):
        for t in range(len(t_set)):
            for d in range(len(d_set)):
                w_fixed[gen_set[g],t_set[t],d_set[d]] = round(model.w[gen_set[g],t_set[t],d_set[d]].value)
                u_fixed[gen_set[g],t_set[t],d_set[d]] = round(model.u[gen_set[g],t_set[t],d_set[d]].value)
                f_fixed[gen_set[g],t_set[t],d_set[d]] = round(model.f[gen_set[g],t_set[t],d_set[d]].value)


    if ess_flag:
        ifch_fixed = {}
        ifdch_fixed = {}


        for s in stor_set:
            for t in t_set:
                for d in d_set:
                    ifch_fixed[s,t,d] = round(model.ifch[s,t,d].value)
                    ifdch_fixed[s,t,d] = round(model.ifdch[s,t,d].value)

    
    
    
    
    
    storage_profit_daily = -1
    

    
    
    
    obj_daily = [sum(
                sum((model.PG[g,t,e].value*GDATA['Cg'][g]+w_fixed[g,t,e]*GDATA['Noload'][g]+
                     u_fixed[g,t,e]*GDATA['StartUp'][g]+f_fixed[g,t,e]*GDATA['ShutDn'][g]) for g in gen_set)+
                sum(model.PLS[b,t,e].value*VOLL for b in bus_set)+
                sum(Curt*model.Pcurt[b,t,e].value for b in bus_set)
            for t in t_set)
                for e in d_set]
    
    
    
    #g2_cost_daily = [sum(
    #        model.PG[2,t,e].value*GDATA['Cg'][2]+w_fixed[2,t,e]*GDATA['Noload'][2]+
    #             u_fixed[2,t,e]*GDATA['StartUp'][2]+f_fixed[2,t,e]*GDATA['ShutDn'][2]
    #    for t in t_set)
    #        for e in d_set]
    
    
    g_cost_daily_lst = []
    for g in gen_set:
        cost_of_g = sum([sum(model.PG[g,t,e].value*GDATA['Cg'][g]+w_fixed[g,t,e]*GDATA['Noload'][g]+ u_fixed[g,t,e]*GDATA['StartUp'][g]+f_fixed[g,t,e]*GDATA['ShutDn'][g] for t in t_set) for e in d_set])
        g_cost_daily_lst.append(cost_of_g)
    
    
                
    wind_curt_cost_daily_per_bus = [0]*len(bus_set)
    load_shed_cost_daily_per_bus = [0]*len(bus_set)
    for id_b, b in enumerate(bus_set):
        wind_curt_cost_daily_per_bus[id_b] = sum(sum(Curt*model.Pcurt[b,t,e].value for t in t_set) for e in d_set)
        load_shed_cost_daily_per_bus[id_b] = sum(sum(VOLL*model.PLS[b,t,e].value for t in t_set) for e in d_set)
    
    
    per_bus_wind_and_load_cost = {}
    per_bus_wind_and_load_cost['wind'] = wind_curt_cost_daily_per_bus
    per_bus_wind_and_load_cost['load'] = load_shed_cost_daily_per_bus
    
    
    
    #wc_cost_daily = [sum(Curt*model.Pcurt[3,t,e].value
    #for t in t_set)
    #    for e in d_set]
    
    
    #obj_cost_breakdown = {}
    #obj_cost_breakdown['g1'] = g1_cost_daily
    #obj_cost_breakdown['g2'] = g2_cost_daily
    #obj_cost_breakdown['w'] = wc_cost_daily

    
    wind_curtailment_daily = [sum(sum(model.Pcurt[b,t,e].value for b in bus_set) for t in t_set) for e in d_set]
    wind_notcurtailed_daily = -1 #[sum(sum(P_wind[b,t,e] - model.Pcurt[b,t,e].value for b in bus_set) for t in t_set) for e in d_set]
    
    
    pwind_curt_detail = np.zeros((len(d_set),len(t_set),len(bus_set))) 
    pwind_generated_detail = np.zeros((len(d_set),len(t_set),len(bus_set)))
    
    for idx_d, d in enumerate(d_set):
        for idx_b, b in enumerate(bus_set):
            for idx_t, t in enumerate(t_set):
                pwind_curt_detail[idx_d, idx_t, idx_b] = model.Pcurt[b, t, d].value
                pwind_generated_detail[idx_d, idx_t, idx_b] = P_wind[b, t, d]
                
                
    re_info_dict={'generated': pwind_generated_detail, 
                  'curtailed': pwind_curt_detail,
                 'format':{'axis0': 'day',
                           'axis1': 'time', 
                           'axis2': 'bus'}
                 }
    
    
    

    gen_power_dict={}
    gen_param_dict={}
    for g in gen_set:
        gen_power_dict[g]=np.zeros((365,24))
        gen_param_dict[g] = {}
        
        for ind_e, e in enumerate(d_set):
            for ind_t, t in enumerate(t_set):
                gen_power_dict[g][ind_e,ind_t] =  model.PG[g,t,e].value
                
                if skip_LMP==True:
                    gen_param_dict[g]['u'] = model.u[g,t,e].value
                    gen_param_dict[g]['f'] = model.f[g,t,e].value
                    gen_param_dict[g]['w'] = model.w[g,t,e].value
                
                else:
                    gen_param_dict[g]['u'] = u_fixed[g,t,e]
                    gen_param_dict[g]['f'] = f_fixed[g,t,e]
                    gen_param_dict[g]['w'] = w_fixed[g,t,e]

    
    
    PLS_dict={}
    for b in bus_set:
        PLS_dict[b]=np.zeros((365,24))
        for ind_e, e in enumerate(d_set):
                for ind_t, t in enumerate(t_set):
                    PLS_dict[b][ind_e,ind_t] =  model.PLS[b,t,e].value
                    if PLS_dict[b][ind_e,ind_t]>0:
                        print(f'there is load shedding at e={e}, bus={b} and time={t}: this_much={PLS_dict[b][ind_e,ind_t]}')
                    
                    tmp = model.Pcurt[b,t,e].value
                    if tmp>0:
                        print(f'there is wind curt at e={e}, bus={b} and time={t}: this_much={tmp} < {P_wind[b,t,e]}=produced ')
    
    
    
    
    
    dict_details={}
    dict_details['gen_power'] = gen_power_dict
    dict_details['gen_param'] = gen_param_dict
    dict_details['load_shed'] = PLS_dict
    

    dict_ess_profile={}    
    if ess_flag:
        storage_ch_daily =  np.zeros((len(d_set),len(t_set),len(stor_set)))
        storage_dch_daily =  np.zeros((len(d_set),len(t_set),len(stor_set)))
        
        for ind_e, e in enumerate(d_set):
            for ind_t, t in enumerate(t_set):
                for ind_s,s in enumerate(stor_set):
                    storage_ch_daily[ind_e,ind_t,ind_s] = model.pch[s,t,e].value
                    storage_dch_daily[ind_e,ind_t,ind_s] = model.pdch[s,t,e].value
                    
        #storage_ch_daily = [sum(sum(model.pch[s,t,e].value for s in stor_set) for t in t_set) for e in d_set]
        #storage_dch_daily = [sum(sum(model.pdch[s,t,e].value for s in stor_set) for t in t_set) for e in d_set]
        storage_op_dict={}
        storage_op_dict['ch'] = storage_ch_daily
        storage_op_dict['dch'] = storage_dch_daily
        
        
        #ch_profile = np.zeros((365,24))
        #for ind_e, e in enumerate(d_set):
        #    for ind_t, t in enumerate(t_set):
        #        ch_profile[ind_e,ind_t] =  SDATA['eff'][1]*model.pch[1,t,e].value
        
        
        #dch_profile = np.zeros((365,24))
        #for ind_e, e in enumerate(d_set):
        #    for ind_t, t in enumerate(t_set):
        #        dch_profile[ind_e,ind_t] =  (1/SDATA['eff'][1])*model.pdch[1,t,e].value
                
                
        #ch_and_dch_profile = ch_profile - dch_profile
        
        
        #dict_ess_profile['ch_profile']=ch_profile
        #dict_ess_profile['dch_profile']=dch_profile
        #dict_ess_profile['ch_and_dch_profile']=ch_and_dch_profile
        
        
        
        soc_profile = np.zeros((365,24,len(stor_set)))
        
        for ind_e, e in enumerate(d_set):
            for ind_t, t in enumerate(t_set):
                for ind_s,s in enumerate(stor_set):
                    soc_profile[ind_e,ind_t,ind_s] =  model.soc[s,t,e].value
        
        
        dict_ess_profile['soc_profile']=soc_profile
        #print('shape of soc_profile: ', np.shape(soc_profile))
        
        
    else:
        
        dict_ess_profile['ch_and_dch_profile']=-1
        dict_ess_profile['soc_profile'] = -1
        
        storage_op_dict={}
        storage_op_dict['ch']=-1
        storage_op_dict['dch']=-1
        ch_profile=-1
    
        
    
    
    
    lineflows = np.zeros((len(d_set),len(t_set),len(lin_set)))
    lineflows_cap = np.zeros(len(lin_set))
    
    for ind_e, e in enumerate(d_set):
        for ind_t, t in enumerate(t_set):
            for ind_lne, lne in enumerate(lin_set):
                lineflows_cap[ind_lne] = LDATA['Flmax'][lne]
                lineflows[ind_e, ind_t, ind_lne] = model.PL[lne,t,e].value / LDATA['Flmax'][lne]
    
    
    
    
    lineflows_fraction  = np.zeros((len(d_set),len(lin_set)))
    for ind_e, e in enumerate(d_set):
        for ind_lne, lne in enumerate(lin_set):
            tmp =  sum(abs(model.PL[lne,t,e].value) for t in t_set)/24
            lineflows_fraction[ind_e, ind_lne] = (1/LDATA['Flmax'][lne]) * tmp
    
    
    
    
    
    #print('storage profit: ', storage_profit)


    #print("\nValue of Total Wind Curtailment (MWh) = ", sum(sum(sum(model.Pcurt[b,t,e].value for b in bus_set) for t in t_set) for e in d_set))

    #print("\nValue of Total Load shedding (MWh) = ", sum(sum(sum(model.PLS[b,t,e].value for b in bus_set) for t in t_set) for e in d_set))

    lines_dict={'lineflows' : lineflows, 'lineflows_cap':lineflows_cap}
    
    return (obj_daily, 
            storage_profit_daily, 
            wind_curtailment_daily, 
            wind_notcurtailed_daily, 
            storage_op_dict, 
            dict_ess_profile, 
            dict_details, 
            #obj_cost_breakdown, 
            g_cost_daily_lst, 
            per_bus_wind_and_load_cost, 
            re_info_dict, 
            lines_dict,
            model,
            lineflows_fraction
           )